# Preparation

In [1]:
import torch
torch.__version__

'2.3.0a0+6ddf5cf85e.nv24.04'

In [2]:
import os
import pandas as pd
import numpy as np
import random
from torch import nn

from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

from model import *
from dataset import *
from train import *

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [4]:
BATCH_SIZE = 256
NUMEPOCHS = 50

In [5]:
# import pandas as pd
# df = pd.read_csv("Data/ml-1m.train.rating",sep='\t', header=None, names=['uid', 'mid', 'rating', 'timestamp'], engine='python')
# df

In [6]:
# df2 = pd.read_csv("Data/ml-1m.test.negative",sep = "\t",header=None)
# df2

# Trian and Test

### Compare the three methods 

In [7]:
model_config_mf = {
    'embedding_dim_mf': 8,
    # "mlp_layers(X)" : 0,
    # 'mlp_layers': [ 32, 16, 8],
    'model_type': 'GMF'     #　MLP, NeuMF
}

model_config_mlp = {
    "mlp_layers(X)" : 3,
    'mlp_layers': [32, 16, 8],
    'model_type': 'MLP'     #　MLP, NeuMF
}

model_config_neumf = {
    'embedding_dim_mf': 8,
    "mlp_layers(X)" : 3,
    'mlp_layers': [32, 16, 8],
    'model_type': 'NeuMF'     #　MLP, NeuMF
}

model_train(model_config_mf, seed = 42)


Using cuda device
Loading preprocessed data from preprocessed_data/ml.pkl
config of model: 
 {'embedding_dim_mf': 8, 'model_type': 'GMF', 'num_users': 6040, 'num_items': 3706}


  5%|▌         | 1/20 [00:12<03:59, 12.61s/it]

Epoch 1, Loss: 0.0027, HR@10: 0.1018, NDCG@10: 0.0450


 10%|█         | 2/20 [00:24<03:36, 12.02s/it]

Epoch 2, Loss: 0.0027, HR@10: 0.0935, NDCG@10: 0.0425


...

Epoch 18, Loss: 0.0002, HR@10: 0.3015, NDCG@10: 0.1511


 95%|█████████▌| 19/20 [03:54<00:12, 12.10s/it]

Epoch 19, Loss: 0.0002, HR@10: 0.2873, NDCG@10: 0.1435


100%|██████████| 20/20 [04:07<00:00, 12.39s/it]

Epoch 20, Loss: 0.0002, HR@10: 0.2763, NDCG@10: 0.1373
Best HR: 0.3574503311258278, Best NDCG: 0.18209659278163892 at Epoch 10


for movielens

HR, NDCG

-   `bias = False` for some linear layers
    -   GMF(Factor-8, X-0): 0.3490, 0.1849
    -   MLP(Factor-8, X-3): 0.4235, 0.2279
    -   NeuMF(Factor-8, X-3): 0.4233, 0.2258

## Reproduce the ablation study --- MLP with different layers

 K: 8
 
 layers (X): 0, 1, 2, 3, 4

In [8]:
# model_config_mlp0 = {
#     "mlp_layers(X)" : 0,
#     'mlp_layers': [8],
#     'model_type': 'MLP'     #　MLP, NeuMF
# }

# model_config_mlp1 = {
#     "mlp_layers(X)" : 1,
#     'mlp_layers': [8],
#     'model_type': 'MLP'     #　MLP, NeuMF
# }

# model_config_mlp2 = {
#     "mlp_layers(X)" : 2,
#     'mlp_layers': [16, 8],
#     'model_type': 'MLP'     #　MLP, NeuMF
# }

# model_config_mlp3 = {
#     "mlp_layers(X)" : 4,
#     'mlp_layers': [32, 16, 8],
#     'model_type': 'MLP'     #　MLP, NeuMF
# }

# model_config_mlp4 = {
#     "mlp_layers(X)" : 4,
#     'mlp_layers': [64, 32, 16, 8],
#     'model_type': 'MLP'     #　MLP, NeuMF
# }

# model_train(model_config_mlp2)


HR, NDCG

-   MLP0: 0.4368, 0.2394
-   MLP1: 0.4368, 0.2394
-   MLP2: 0.4222, 0.2244
-   MLP3: 